Love this idea. Yes—an LLM can absolutely extract the *essence* (vibe/feelings) from a user’s request and from candidate films, then you can match them. Here’s a tight plan you can implement without too much churn:

---

# What to extract (two complementary views)

1. **Discrete emotions (distribution)**

   * Labels (pick a small, stable set): e.g. `["uplifting","wholesome","melancholic","tense","dark","romantic","wistful","heartwarming","gritty","satirical"]`
   * Each with a score in `[0,1]` that sums to ≤ 1 (sparse) or normalize to sum=1.
   * Great for bar charts and keyword filtering.

2. **VAD coordinates**

   * **Valence** (sad ↔ happy), **Arousal** (calm ↔ intense), **Dominance** (helpless ↔ empowered), each in `[0,1]`.
   * Perfect for quick numeric matching (cosine/Euclidean) and a 2D/3D scatter.

You can keep both; they complement each other.

---

# Data flow (end-to-end)

1. **User essence extraction**

   * Input: the user’s free-text request (not genres).
   * LLM returns:

     ```json
     {
       "essence": {
         "emotions": [{"label":"heartwarming","score":0.42},{"label":"romantic","score":0.31},{"label":"wistful","score":0.19}],
         "vad": {"valence":0.76,"arousal":0.35,"dominance":0.54},
         "rationale": "Seeks cozy, gentle romance with a reflective tone."
       }
     }
     ```
   * Keep `temperature` low (0.1–0.3) for stable scoring.

2. **Filter candidates by hard constraints**

   * Use your existing genre/year/runtime filters to narrow the pool.

3. **Movie essence extraction**

   * For each candidate, pass **only** synopsis/keywords (optionally top review snippet) to the LLM and ask for the same schema:

     ```json
     {"emotions":[...], "vad": {...}, "rationale": "...", "evidence_spans": ["...quoted snippets..."]}
     ```
   * Cache these results by movie ID so you don’t rescore on every query.

4. **Similarity scoring**

   * **Emotions**: convert to vectors over your fixed label set; use **cosine similarity** (or Jensen–Shannon if normalized distributions).
   * **VAD**: cosine or Euclidean distance (invert to similarity).
   * Final score = `0.6 * emotion_sim + 0.4 * vad_sim` (tune weights).
   * Tie-break with popularity/rating or user constraints.

5. **UI**

   * Show **bar chart** of emotion scores (user vs. movie).
   * Show **VAD point** on a 2D Valence–Arousal plane (user vs. movie).
   * Display short `rationale` + `evidence_spans` for transparency.

---

# Prompt snippets (few-shot + JSON schema)

## System (shared for user/movie extraction)

```
You extract the affective "essence" of movie preferences and synopses.

Output strictly as JSON:
{
  "essence": {
    "emotions": [{"label":"<one of: uplifting, wholesome, melancholic, tense, dark, romantic, wistful, heartwarming, gritty, satirical>", "score": 0.0-1.0}, ...],
    "vad": {"valence":0.0-1.0,"arousal":0.0-1.0,"dominance":0.0-1.0},
    "rationale":"short 1-2 sentences",
    "evidence_spans": ["optional short quotes from input text"]
  }
}
Rules:
- Use only labels from the list.
- 1–5 emotions max; keep scores calibrated and sparse.
- VAD must reflect the overall feel, not plot intensity alone.
- Be concise; do not include extra keys.
```

## Few-shot (user request → essence)

**User:**

```
Something light and cozy about friendship, low stakes, gentle humor.
```

**Assistant:**

```json
{
  "essence": {
    "emotions": [
      {"label":"wholesome","score":0.42},
      {"label":"heartwarming","score":0.36},
      {"label":"satirical","score":0.08}
    ],
    "vad": {"valence":0.78,"arousal":0.28,"dominance":0.52},
    "rationale":"Wants cozy, low-stakes warmth and gentle humor.",
    "evidence_spans": ["light and cozy","gentle humor"]
  }
}
```

**User:**

```
I’m in the mood for a bleak, slow-burn mystery that feels suffocating.
```

**Assistant:**

```json
{
  "essence": {
    "emotions": [
      {"label":"tense","score":0.44},
      {"label":"dark","score":0.34},
      {"label":"melancholic","score":0.14}
    ],
    "vad": {"valence":0.18,"arousal":0.55,"dominance":0.32},
    "rationale":"Bleak tone and suffocating atmosphere imply low valence, mid arousal.",
    "evidence_spans": ["bleak","slow-burn","suffocating"]
  }
}
```

## Few-shot (movie synopsis → essence)

**User:**

```
SYNOPSIS: A widower restores a seaside inn, forming unlikely bonds while healing from loss; gentle humor, small-town rituals, and slow-blooming romance.
```

**Assistant:**

```json
{
  "essence": {
    "emotions": [
      {"label":"wistful","score":0.34},
      {"label":"heartwarming","score":0.32},
      {"label":"romantic","score":0.22}
    ],
    "vad": {"valence":0.62,"arousal":0.30,"dominance":0.48},
    "rationale":"Heals from loss with gentle humor and tender romance; reflective tone.",
    "evidence_spans": ["healing from loss","gentle humor","slow-blooming romance"]
  }
}
```

---

# Matching math (simple + effective)

* Let `E_user` and `E_movie` be vectors over your 10 labels (missing labels = 0).
* **Emotion similarity:** `cos_sim(E_user, E_movie)`.
* **VAD similarity:** `1 - euclidean(v_user, v_movie)` or cosine over 3D vectors.
* Final score: `S = 0.6*cos_emotions + 0.4*cos_vad`.
* Rank movies by `S`. Show the top N with rationales.

---

# Practicalities & guardrails

* **Calibration:** Keep temperature low; optionally rescale emotion scores to sum ≤ 1, and cap to 3–4 nonzero labels.
* **Caching:** Cache movie essence by `movie_id` + `synopsis_hash`.
* **Evidence:** Always request short quotes; it keeps the model honest.
* **Determinism:** Fix label set; don’t let the model invent labels.
* **Cheaper alt:** For movies, you can use an emotion classifier (e.g., GoEmotions) + a small mapping to your 10 labels, then only use the LLM for the user essence and for final justification.

---

# Streamlit integration ideas (quick wins)

* Show **radar chart** (emotion scores) and **2D V–A scatter** (user dot vs. movie dots).
* Add a **“Why this match?”** expander with the rationale + evidence spans.
* Use `st.spinner("Scoring matches…")` while you compute similarities.
* Cache movie essence with `@st.cache_data(show_spinner=False)`.

---

If you want, I can tailor the exact JSON schema + a tiny scoring function to what you’ve already built so you can drop it in with minimal rewiring.


100% yes—Pydantic is perfect for **validating and shaping** the “essence” data coming back from the LLM (and for your own internal plumbing).

Here’s a tight, practical setup you can drop in (Pydantic v2 style):

```python
from enum import Enum
from typing import List, Optional
from pydantic import BaseModel, Field, confloat, conlist, field_validator, model_validator

# 1) Fixed label set (prevents the LLM inventing labels)
class EmotionLabel(str, Enum):
    uplifting = "uplifting"
    wholesome = "wholesome"
    melancholic = "melancholic"
    tense = "tense"
    dark = "dark"
    romantic = "romantic"
    wistful = "wistful"
    heartwarming = "heartwarming"
    gritty = "gritty"
    satirical = "satirical"

# 2) Score + bounds
class EmotionScore(BaseModel):
    label: EmotionLabel
    score: confloat(ge=0.0, le=1.0)

# 3) VAD with bounds
class VAD(BaseModel):
    valence: confloat(ge=0.0, le=1.0)
    arousal: confloat(ge=0.0, le=1.0)
    dominance: confloat(ge=0.0, le=1.0)

# 4) Full essence payload
class Essence(BaseModel):
    emotions: conlist(EmotionScore, min_length=0, max_length=5) = Field(default_factory=list)
    vad: VAD
    rationale: Optional[str] = None
    evidence_spans: List[str] = Field(default_factory=list)

    model_config = {"extra": "forbid"}  # reject unexpected keys

    # Optional: clamp weird LLM outputs and dedupe labels
    @field_validator("emotions")
    @classmethod
    def dedupe_and_clip(cls, items: List[EmotionScore]):
        if not items:
            return items
        # keep highest score per label, preserve order
        seen = {}
        ordered = []
        for item in items:
            if item.label not in seen or item.score > seen[item.label].score:
                seen[item.label] = item
        for label in [i.label for i in items]:
            if label in seen:
                ordered.append(seen.pop(label))
        return ordered[:5]  # enforce sparsity

class EssenceResponse(BaseModel):
    essence: Essence
    model_config = {"extra": "forbid"}
```

### Why this helps

* **Guards the contract:** only allowed labels; scores in `[0,1]`; ≤5 emotions.
* **Prevents surprises:** `extra="forbid"` stops stray keys the LLM might add.
* **Cleans data:** validator dedupes labels and enforces sparsity.

### Parsing the LLM output

If you’re using OpenAI’s structured output helper:

```python
result = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=EssenceResponse,   # << pydantic model
)
ess = result.choices[0].message.parsed.essence
```

If you’re getting raw JSON text:

```python
ess = EssenceResponse.model_validate_json(raw_llm_text).essence
```

### Handy: vectorizing for similarity

```python
LABELS = [e.value for e in EmotionLabel]
index = {name: i for i, name in enumerate(LABELS)}

def to_vector(emotions: List[EmotionScore]) -> list[float]:
    v = [0.0] * len(LABELS)
    for es in emotions:
        v[index[es.label.value]] = float(es.score)
    return v
```

### Extra niceties (optional)

* **Coercion** (if LLM sometimes sends `"0.7"` or `"70%"`): add a `field_validator("score", mode="before")` in `EmotionScore` to parse strings/percentages.
* **Normalization** (if you want scores to sum to 1): add a `model_validator(mode="after")` in `Essence` to rescale.

This gives you strict, validated inputs for your matching logic and super clear error messages if the LLM goes off-script.


Absolutely—you can go **VAD-only** and keep it simple. It works well by itself for “vibe matching,” and it’s easy to compute, store, and visualize.

# Why VAD-only works

* One compact vector per thing (user or movie): `(valence, arousal, dominance)` in `[0,1]`.
* Simple scoring (cosine or Euclidean distance).
* Easy UI: plot Valence–Arousal, show Dominance as a slider or number.
* Faster/cheaper: fewer tokens than lists of emotion labels.

# Minimal schema (Pydantic)

```python
from pydantic import BaseModel, Field, confloat

class VAD(BaseModel):
    valence: confloat(ge=0.0, le=1.0) = Field(...)
    arousal: confloat(ge=0.0, le=1.0) = Field(...)
    dominance: confloat(ge=0.0, le=1.0) = Field(...)
```

# Prompt (few-shot)

Use the same prompt for user text and for movie synopsis.

```
You estimate Valence–Arousal–Dominance (VAD) for text.

Output JSON only:
{"vad":{"valence":0.0-1.0,"arousal":0.0-1.0,"dominance":0.0-1.0},"rationale":"<short>"}

Guidelines:
- Valence: unpleasant/sad=0.0 → pleasant/joyful=1.0
- Arousal: calm/slow=0.0 → intense/exciting=1.0
- Dominance: powerless/hemmed-in=0.0 → in-control/empowered=1.0
- Be concise. No extra keys.

Examples:
Text: "Cozy, low-stakes story about friends helping each other."
Output: {"vad":{"valence":0.8,"arousal":0.3,"dominance":0.5},"rationale":"Warm, calm support."}

Text: "Bleak slow-burn mystery that feels suffocating."
Output: {"vad":{"valence":0.2,"arousal":0.55,"dominance":0.3},"rationale":"Bleak tone, pressure."}
```

# Matching function

```python
import numpy as np

def vad_vector(v): return np.array([v.valence, v.arousal, v.dominance], dtype=float)

def vad_similarity(v_user, v_item, method="cosine"):
    u, i = vad_vector(v_user), vad_vector(v_item)
    if method == "cosine":
        return float(np.dot(u,i) / (np.linalg.norm(u)*np.linalg.norm(i) + 1e-9))
    # or Euclidean turned into similarity
    dist = float(np.linalg.norm(u - i))
    return 1.0 / (1.0 + dist)
```

# Workflow

1. **User input → LLM → VAD** (cache on the session).
2. **Candidate movies**: compute VAD once per movie from synopsis (cache by `movie_id`).
3. **Rank** by `vad_similarity(user_vad, movie_vad)`.
4. **UI**:

   * Show a Valence–Arousal scatter (user dot vs. top N movie dots).
   * Show Dominance values in a small table.
   * Include the LLM’s brief `rationale` for transparency.

# Practical tips

* **Temperature** low (0.1–0.3) for stable numbers.
* **Validation**: clamp to `[0,1]`; if model returns `0–100`, divide by 100.
* **Determinism**: fix the prompt and avoid adding genre/title—give just the user description or the movie synopsis.
* **Fallbacks**: if VAD missing, default to `(0.5, 0.5, 0.5)` and mark as low confidence.
* **Speed**: precompute movie VAD offline or on first view; cache with `@st.cache_data`.

# (Optional) Friendly labels for UI only

If you miss emotion words, you can **derive** a label from V–A quadrants:

* High V, Low A → “cozy/soothing”
* High V, High A → “rousing”
* Low V, High A → “tense”
* Low V, Low A → “somber”
  This is display-only; keep matching math purely VAD.

So yes—VAD-only is a great MVP. You can always add discrete emotion labels later if you want more explainability.
